http://www.sports-reference.com/cbb/postseason/1939-ncaa.html

In [50]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import re

def getPageData(year):
    url = 'http://www.sports-reference.com/cbb/postseason/'+str(year)+'-ncaa.html'
    page = urlopen(url)
    return BeautifulSoup(page, 'html.parser')

soup = getPageData(2016)
east_region_html = soup.select("div#east div#bracket div:nth-child(1)")
len(east_region_html)
for div in east_region_html:
    print(div)
    print()


NotImplementedError: Only the following pseudo-classes are implemented: nth-of-type.

[]